In [8]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from operations import adjust_dataset, split_data, Ann, adjust_dataset2


In [9]:
df = pd.read_csv('docs/data/GlobalWeatherRepository.csv')
target_variable = 'temperature_celsius'
df = adjust_dataset(df, target_variable=target_variable)
print (df.shape)
# for column in df.columns:
#     # Print the mean value of columns
#     print(column, df[column].mean())

# lets print all the values (just print the name of the column) whose
# value is higher than 1 or lower than -1. Do not include temperature_celsius


# for column in df.columns:
#     # if column is not the target variable and is an integer or float
#     if column != target_variable and df[column].dtype in ['int64', 'float64']:
#         # print the column name
#         print(column)
#         # print the values that are higher than 1 or lower than -1
#         print(df[column][(df[column] > 1) | (df[column] < -1)])
#         print('\n')

(17946, 48)


In [12]:
df = df[:-7]
model = Ann(df, 'temperature_celsius')
model.train_model(epochs=50, batch_size=32)

Cantidad de windows:  15962
Cantidad de labels:  15962


In [13]:
res = model.evaluate()
print (res)

3.349555730819702


In [ ]:

# Compute feature importance
feature_importance_values = model.compute_feature_importance()

sorted_feature_importance = sorted(feature_importance_values, key=lambda x: x[1], reverse=True)

# Iterate over the sorted list and print column name and value
for column, value in sorted_feature_importance:
    print(f"{column} --> {value}")


gust_kph --> 1.0253242521685024e-07
air_quality_PM10 --> 1.0253242521685024e-07
sunset --> 1.0253242521685024e-07
moon_illumination --> 1.0253242521685024e-07
condition_text_Fog --> 1.0253242521685024e-07
moon_phase_Last Quarter --> 1.0253242521685024e-07
moon_phase_Waning Gibbous --> 1.0253242521685024e-07
wind_kph --> 0.0
wind_degree --> 0.0
pressure_mb --> 0.0
precip_mm --> 0.0
humidity --> 0.0
cloud --> 0.0
feels_like_celsius --> 0.0
visibility_km --> 0.0
uv_index --> 0.0
air_quality_Carbon_Monoxide --> 0.0
air_quality_Ozone --> 0.0
air_quality_Nitrogen_dioxide --> 0.0
air_quality_Sulphur_dioxide --> 0.0
air_quality_PM2.5 --> 0.0
sunrise --> 0.0
moonrise --> 0.0
moonset --> 0.0
condition_text_Clear --> 0.0
condition_text_Light rain --> 0.0
condition_text_Mist --> 0.0
condition_text_Moderate or heavy rain with thunder --> 0.0
condition_text_Overcast --> 0.0
condition_text_Partly cloudy --> 0.0
condition_text_Sunny --> 0.0
wind_direction_E --> 0.0
wind_direction_ENE --> 0.0
wind_dire

In [ ]:
model.predict(df[-7:])

array([14.498802, 13.253254, 12.907271], dtype=float32)

# Hyperparameter tuning


In [ ]:
hyperparameter_grid = {
    'lr': [0.01],
    'batch_size': [32, 64],
    'dropout': [0.2],
    'num_layers': [1, 2, 3],
    'neurons_per_layer': [32, 64, 128],
    'lr_schedule': ['constant', 'step_decay', 'exponential_decay'],
    'optimizer': ['SGD', 'Adam', 'RMSprop']
}
num_epochs_list = [50, 100]
tuning_results = model.hyperparameter_tuning(hyperparameter_grid, num_epochs_list=num_epochs_list)

print (tuning_results)